In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [4]:
##FFmpg (turns your gource into a MP4)
#brew install ffmpeg

In [5]:
##when that is finished installing type 
#brew install gource

**Run this with**:
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log --seconds-per-day 1 -1440x900 --bloom-intensity 0.1 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide filenames

**Output video**:
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide dirnames, -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 output.mp4

- link : [custom log format](https://github.com/acaudwell/Gource/wiki/Custom-Log-Format)
- link : [visualization](https://edgeryders.eu/t/visualizing-our-7-years-of-online-dialogue-with-gource/11905)
- likn : [stuff to gource](https://leonjza.github.io/blog/2013/07/27/stuff-to-gource./#get-the-code-setup-your-own)

The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [2]:
import pandas as pd
import pandasql as ps
import numpy as np
import re
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
df_characters = pd.read_csv('../df_characters.csv')
df_episodes = pd.read_csv('../df_episodes.csv')
df_transcripts = pd.read_csv('../df_transcripts.csv')
df_rel_sent = pd.read_csv('../df_rel_sent.csv')

In [4]:
from ast import literal_eval
df_characters['episode'] = df_characters['episode'].apply(literal_eval)

In [5]:
df_characters_exploded = df_characters.explode('episode')

In [6]:
df_characters_exploded.episode = df_characters_exploded.episode.astype(int)

In [7]:
df_characters_exploded

,name,status,species,type,gender,origin,location,image,episode,url,created,references
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,1,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,2,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,3,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,4,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,5,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
821,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z,NaN
822,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z,NaN
823,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z,NaN
824,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z,NaN


In [8]:
import datetime
from datetime import datetime
from datetime import timedelta
import time

In [9]:
date_range = pd.date_range(datetime.today() - timedelta(days=51), periods=51)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series).values[::-1])\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp


,episode,timestamp,unixtime
0,1,2021-11-30 09:58:37.607286,1638266317607286000
1,2,2021-11-29 09:58:37.607286,1638179917607286000
2,3,2021-11-28 09:58:37.607286,1638093517607286000
3,4,2021-11-27 09:58:37.607286,1638007117607286000
4,5,2021-11-26 09:58:37.607286,1637920717607286000
5,6,2021-11-25 09:58:37.607286,1637834317607286000
6,7,2021-11-24 09:58:37.607286,1637747917607286000
7,8,2021-11-23 09:58:37.607286,1637661517607286000
8,9,2021-11-22 09:58:37.607286,1637575117607286000
9,10,2021-11-21 09:58:37.607286,1637488717607286000


In [165]:
date_range = pd.date_range(datetime.today() - timedelta(days=19), periods=19)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [166]:
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-12 11:26:41.576422,1636716401
1,2,2021-11-13 11:26:41.576422,1636802801
2,3,2021-11-14 11:26:41.576422,1636889201
3,4,2021-11-15 11:26:41.576422,1636975601
4,5,2021-11-16 11:26:41.576422,1637062001
5,6,2021-11-17 11:26:41.576422,1637148401
6,7,2021-11-18 11:26:41.576422,1637234801
7,8,2021-11-19 11:26:41.576422,1637321201
8,9,2021-11-20 11:26:41.576422,1637407601
9,10,2021-11-21 11:26:41.576422,1637494001


The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [13]:
df_characters_exploded

,name,status,species,type,gender,origin,location,image,episode,url,created,references
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,1,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,2,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,3,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,4,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,5,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
821,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z,NaN
822,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z,NaN
823,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z,NaN
824,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z,NaN


# Character pr. episode (custom_char_episode.log)

- 1633360391|episode_1|A|episode_1/rick/rick.cpp
- 1633360391|episode_1|A|episode_1/morty/rick.cpp
- 1633360391|episode_1|A|episode_1/summer/rick.cpp
- 1633360391|episode_2|A|episode_2/rick.cpp
- 1633360391|episode_3|A|episode_3/rick.cpp

In [14]:
res = ps.sqldf(
    """
    select 
        1633360391 || '|' || 'episode_' || episode || '|' || 'A' || '|' || 'episode_' || episode || '/' || character || '/' || 'player.cpp' as output
    from
    
    (
    
    select 
       episode
    ,  character
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
        from df_characters_exploded df
        order by df.episode asc
        )
    order by episode asc
    )
    """
)

res

,output
0,1633360391|episode_1|A|episode_1/Rick_Sanchez/...
1,1633360391|episode_1|A|episode_1/Morty_Smith/p...
2,1633360391|episode_1|A|episode_1/Bepisian/play...
3,1633360391|episode_1|A|episode_1/Beth_Smith/pl...
4,1633360391|episode_1|A|episode_1/Canklanker_Th...
...,...
1262,1633360391|episode_51|A|episode_51/Redhead_Mor...
1263,1633360391|episode_51|A|episode_51/Young_Jerry...
1264,1633360391|episode_51|A|episode_51/Young_Beth/...
1265,1633360391|episode_51|A|episode_51/Young_Beth/...


In [15]:
with open('custom_char_episode.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')

# Episode, Sentiment pr Character (top 5 pr episode)

The idea about this gource is to:
- Episodes are branches to eachother (linked)
- For each episode, take the top 5 (total sentences) characters
- Color the character according to sentiment

In [33]:
res = ps.sqldf(
    """
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    
    where rn <= 5
    """
)

- `1275543595|matthias|A|earthos/reef/1234/1`

In [80]:
res.head(10)

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Annie,0.8002,4
4,Anatomy Park,Dr. Bloom,0.0000,5
5,Auto Erotic Assimilation,Rick,0.5279,1
6,Gotron Jerrysis Rickvangelion,Rick,0.9944,1
7,Gotron Jerrysis Rickvangelion,Summer,0.9942,2
8,Gotron Jerrysis Rickvangelion,Morty,0.6733,3
9,Gotron Jerrysis Rickvangelion,Beth,0.8852,4


800020


In [119]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 0#255 - b - r
    return r, g, b


def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [124]:
rgb_to_hex(rgb(1, -1, -1))

'ff0000'

In [178]:
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-12 11:26:41.576422,1636716401
1,2,2021-11-13 11:26:41.576422,1636802801
2,3,2021-11-14 11:26:41.576422,1636889201
3,4,2021-11-15 11:26:41.576422,1636975601
4,5,2021-11-16 11:26:41.576422,1637062001
5,6,2021-11-17 11:26:41.576422,1637148401
6,7,2021-11-18 11:26:41.576422,1637234801
7,8,2021-11-19 11:26:41.576422,1637321201
8,9,2021-11-20 11:26:41.576422,1637407601
9,10,2021-11-21 11:26:41.576422,1637494001


In [173]:
df_episode_unix_timestamp.loc[0].unixtime

1636716401

In [182]:
with open('episode_character_sentiment.log', 'w') as f:
        

    n = 0
    for idx, row in res.iterrows():
        #print('1633360391' + '|' + str(row.episode))
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            timestamp = str(df_episode_unix_timestamp.iloc[idx + 1].unixtime)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char + '.txt')
                #print(res[(res.episode == episode) & (res.character == char)].sentiment.values[0])
                sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                #print(sentiment_to_hex)
                #print(episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper())
                add_file = episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper()
                print(timestamp + '|' + 'sentiment|A|' + add_file)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')

        if idx != 0:
            episode = res.iloc[idx].episode  
            timestamp = str(df_episode_unix_timestamp.iloc[idx + 1].unixtime)
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                if n == 0:
                    n += 1
                    continue
                #print('idx :', episode)
                episode_gathered += '/' + res.iloc[idx-1].episode
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                    sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                    add_file = episode_gathered + '/' + char + '.txt' + '|' + sentiment_to_hex.upper()
                    print(timestamp + '|' + 'sentiment|A|' + add_file)
                folder = episode_gathered + '/'
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                break

1636802801|sentiment|A|Anatomy Park/Rick.txt|0000CF
1636802801|sentiment|A|Anatomy Park/Morty.txt|920000
1636802801|sentiment|A|Anatomy Park/Jerry.txt|0000F5
1636802801|sentiment|A|Anatomy Park/Annie.txt|0000CC
1636802801|sentiment|A|Anatomy Park/Dr. Bloom.txt|000000
1637321201|sentiment|A|Anatomy Park/Auto Erotic Assimilation/Rick.txt|0000FD
1637321201|sentiment|A|Anatomy Park/Auto Erotic Assimilation/Summer.txt|0000FD
1637321201|sentiment|A|Anatomy Park/Auto Erotic Assimilation/Morty.txt|0000AB
1637321201|sentiment|A|Anatomy Park/Auto Erotic Assimilation/Beth.txt|0000E1
1637321201|sentiment|A|Anatomy Park/Auto Erotic Assimilation/Jerry.txt|0000A8


In [117]:
import math

def num_to_rgb(val, max_val=1):
    i = (val * 255 / max_val);
    r = round(math.sin(0.024 * i + 0) * 127 + 128);
    g = 0# round(math.sin(0.024 * i + 2) * 127 + 128);
    b = round(math.sin(0.024 * i + 4) * 127 + 128);
    return (r,g,b)

def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

print(rgb_to_hex(num_to_rgb(0)))

800020


In [30]:
res = ps.sqldf(
    """
    select 
        1633360391 || '|' || 'sentiment' || '|' || 'A' || '|' || episode as output
    from
    (
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    
    where rn <= 5
    )
    """
)

res

,output
0,1633360391|sentiment|A|Anatomy Park
1,1633360391|sentiment|A|Anatomy Park
2,1633360391|sentiment|A|Anatomy Park
3,1633360391|sentiment|A|Anatomy Park
4,1633360391|sentiment|A|Anatomy Park
...,...
106,1633360391|sentiment|A|Vindicators 3: The Retu...
107,1633360391|sentiment|A|Vindicators 3: The Retu...
108,1633360391|sentiment|A|Vindicators 3: The Retu...
109,1633360391|sentiment|A|Vindicators 3: The Retu...


In [31]:
with open('episode_character_sentiment.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')

# Something Else

In [195]:
ps.sqldf(
    """
    select
          replace(df.name, ' ', '_') as character
        , df.episode
        , ts.timestamp
        , ts.unixtime
    from df_characters_exploded df
    inner join df_episode_unix_timestamp as ts
        on ts.episode = df.episode
    order by df.episode asc
    """
)

,character,episode,timestamp,unixtime
0,Rick_Sanchez,1,2021-10-04 15:13:11.890278,1633360391
1,Morty_Smith,1,2021-10-04 15:13:11.890278,1633360391
2,Bepisian,1,2021-10-04 15:13:11.890278,1633360391
3,Beth_Smith,1,2021-10-04 15:13:11.890278,1633360391
4,Canklanker_Thom,1,2021-10-04 15:13:11.890278,1633360391
...,...,...,...,...
1262,Redhead_Morty,51,2021-11-23 15:13:11.890278,1637680391
1263,Young_Jerry,51,2021-11-23 15:13:11.890278,1637680391
1264,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391
1265,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391


`unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output`

In [214]:
# which characters touch which episode
res = ps.sqldf(
    """
    select 
        unixtime || '|' || character || '|' || 'A' || '|' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|Rick_Sanchez|A|1.txt
1,1633360391|Morty_Smith|A|1.txt
2,1633360391|Bepisian|A|1.txt
3,1633360391|Beth_Smith|A|1.txt
4,1633360391|Canklanker_Thom|A|1.txt
...,...
1262,1637680391|Redhead_Morty|A|51.txt
1263,1637680391|Young_Jerry|A|51.txt
1264,1637680391|Young_Beth|A|51.txt
1265,1637680391|Young_Beth|A|51.txt


In [215]:
#res = ps.sqldf(
#    """
#    
#    select 
#       unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output
#    from 
#    (
#
#    select
#          replace(df.name, ' ', '_') as character
#        , df.episode
#        , ts.timestamp
#        , ts.unixtime
#    from df_characters_exploded df
#    inner join df_episode_unix_timestamp as ts
#        on ts.episode = df.episode
#    )
#    order by 1 desc
#    """
#)

In [219]:
with open('custom.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')
        

- gource --hide dirnames,filenames --seconds-per-day 0.1 --auto-skip-seconds 1 -1280x720 -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -pix_fmt yuv420p -crf 1 -threads 0 -bf 0 gource.mp4

In [223]:
# which episodes that touch the characters
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez.txt
1,1633360391|1|A|Morty_Smith.txt
2,1633360391|1|A|Bepisian.txt
3,1633360391|1|A|Beth_Smith.txt
4,1633360391|1|A|Canklanker_Thom.txt
...,...
1262,1637680391|51|A|Redhead_Morty.txt
1263,1637680391|51|A|Young_Jerry.txt
1264,1637680391|51|A|Young_Beth.txt
1265,1637680391|51|A|Young_Beth.txt


In [224]:
# something
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '/' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez/1.txt
1,1633360391|1|A|Morty_Smith/1.txt
2,1633360391|1|A|Bepisian/1.txt
3,1633360391|1|A|Beth_Smith/1.txt
4,1633360391|1|A|Canklanker_Thom/1.txt
...,...
1262,1637680391|51|A|Redhead_Morty/51.txt
1263,1637680391|51|A|Young_Jerry/51.txt
1264,1637680391|51|A|Young_Beth/51.txt
1265,1637680391|51|A|Young_Beth/51.txt
